In [1]:
import PyPDF2
import re
import pandas as pd
import os
from datetime import date
        

In [ ]:

# Keywords to search for (case-insensitive)
KEYWORDS = ['cannot', 'error', 'fail', 'contact broker', 'contact two three bird', 
            'POLICY NOT ADDED', 'POLICY NOT CHANGED', 'POLICY PAYMENT ERROR',
            'INVALID STATUS TO CANCEL', 'DUPLICATE CARRIER POL#', 'INVALID TRAN TYPE:',
            'MODIFICATION NOT ALLOWED', 'CARRIER POLICY NOT FOUND', 'COVERAGE CODE NOT RECEIVED. POLICY IGNORED', 
            'PAYMENT NOT RECEIVED ON INSTALLMENT POLICY', 'PAYMENT RECEIVED ON INVALID RECORD TYPE', 'EARNINGS RECORD RECEIVED FOR NON INSTALLMENT POLICY']

#Datetime variable
today = date.today()


# Policy number pattern: 3 uppercase letters + 2 digits + 5 digits
POLICY_PATTERN = r'\b[A-Z]{3}\d{7}\b'

def extract_relevant_data_from_pdf(pdf_path: str) -> list:
    """Extract relevant data from a single PDF file."""
    with open(pdf_path, 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf, strict=False)
        extracted_data = {}

        for page_num, page in enumerate(reader.pages, start=1):
            content = page.extract_text()
            if not content:
                continue

            # Find matching keywords
            matched_keywords = {kw for kw in KEYWORDS if re.search(rf'\b{kw}\b', content, re.IGNORECASE)}

            # Find policy number
            policy_match = re.search(POLICY_PATTERN, content)
            policy_number = policy_match.group() if policy_match else "Not Found"

            if matched_keywords:
                key = (pdf_path, page_num, policy_number)  # Unique key to prevent duplicates
                if key in extracted_data:
                    extracted_data[key].update(matched_keywords)  # Merge keywords
                else:
                    extracted_data[key] = matched_keywords

        # Convert dictionary to list format
        formatted_data = [(os.path.basename(pdf_path), page_num, ", ".join(sorted(keywords)), policy_number) 
                          for (pdf_path, page_num, policy_number), keywords in extracted_data.items()]
        return formatted_data


def process_all_pdfs_in_folder(folder_path: str, output_file = f"{today}_Filtered_Errors.xlsx"):
    """Process all PDF files in the specified folder and export results to Excel."""
    all_data = []

    # Iterate through all PDF files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            extracted_data = extract_relevant_data_from_pdf(pdf_path)
            all_data.extend(extracted_data)

    if not all_data:
        print("No matching data found in any PDF.")
        return

    # Create DataFrame and export to Excel
    df = pd.DataFrame(all_data, columns=["Filename", "Page Number", "Keywords Found", "Policy Number"])
    df.to_excel(output_file, index=False)
    print(f"Extracted data exported to {output_file}")

if __name__ == '__main__':
    folder_path = 'PDF_Files' 
    process_all_pdfs_in_folder(folder_path)
